In [ ]:
import cvxpy as cp
import pandas as pd
import numpy as np
import csv
from scipy.linalg import eigh
from scipy.linalg import eigvals
from scipy.sparse import coo_matrix
from scipy.optimize import minimize_scalar
from scipy.optimize import root_scalar
import cvxpy as cp
import time
from google.colab import files
from scipy.stats import norm
from numpy.random import randint, choice

# Parameters
p = 50  # number of atoms
m = 25  # dimension of data
n_old = 250  # number of lower-level samples
n_new = 250  # number of upper-level samples
sigma = 0.01  # noise level
max_time = 100  # max running time

seed = 1
np.random.seed(seed)

figs = True

# True dictionary Dstar
Dstar = np.random.randn(m, p)
Dstar = Dstar / np.linalg.norm(Dstar, axis=0)  # Normalize the atoms

k_spar = 5  # number of nonzeros in each coefficient vector

# Old dataset (X_old and A_old)
mask = np.zeros((p, n_old))
for i in range(n_old):
    indices = choice(np.arange(round(4*p/5)), k_spar, replace=False)
    mask[indices, i] = (0.8 * np.random.rand(k_spar) + 0.2) * (2 * randint(0, 2, k_spar) - 1)

X_old = mask  # true coefficient matrix for the old dataset
A_old = Dstar @ X_old + sigma * np.random.randn(m, n_old)

# New dataset (X_new and A_new)
mask = np.zeros((p, n_new))
for i in range(n_new):
    indices = choice(np.arange(round(3*p/5), p), k_spar, replace=False)
    mask[indices, i] = (0.8 * np.random.rand(k_spar) + 0.2) * (2 * randint(0, 2, k_spar) - 1)

X_new = mask  # true coefficient matrix for the new dataset
A_new = Dstar @ X_new + sigma * np.random.randn(m, n_new)



In [ ]:
A_old = pd.read_csv('A_old.csv', header=None)
A_old = A_old.to_numpy()
A_new = pd.read_csv('A_new.csv', header=None)
A_new = A_new.to_numpy()
X_base = pd.read_csv('X_base.csv', header=None)
X_base = X_base.to_numpy()

In [ ]:
D = cp.Variable((m,p))
inner = cp.Problem(cp.Minimize(cp.sum([cp.norm(A_old[:, j] - D @ X_base[:, j], 2)**2 for j in range(n_old)])/(2*n_old)), [cp.norm(D[:, j]) <= 1 for j in range(p)])
inner.solve()
g_opt = inner.value

In [ ]:
delta = 3
def g(D, S):
    diff = A_old[:, S] - D @ X_base[:, S]  # Difference matrix (m x len(S))
    return np.sum(np.linalg.norm(diff, axis=0)**2) / (2 * len(S))

def gradg(D, S):
    X_selected = X_base[:, S]
    return (D @ X_selected - A_old[:, S])@X_selected.T / len(S)

def f(X,D,S):
    diff = A_new[:, S] - D @ X[:, S]  # Difference matrix (m x len(S))
    return np.sum(np.linalg.norm(diff, axis=0)**2) / (2 * len(S))

def gradXf(X,D,S):
    gradXf = np.zeros_like(X)
    X_sel = X[:, S]
    A_sel = A_new[:, S]
    gradXf[:, S] = D.T @ (D @ X_sel - A_sel)
    return gradXf

def gradDf(X,D,S):
    X_selected = X[:, S]
    diff = D @ X_selected - A_new[:, S]
    gradDf = diff@X_selected.T/len(S)
    return gradDf

def lmoX(C1):
  V1 = np.zeros_like(C1)
  for j in range(n_new):
      idx = np.argmax(np.abs(C1[:, j]))  # Find the index of the max gradient
      V1[idx, j] = -delta * np.sign(C1[idx, j])

  return V1

def lmoD(C2):
  norms = np.linalg.norm(C2, axis=0)
  V2 = np.divide(-C2, norms, where=(norms > 0))
  return V2

def simplex_projection(s):
  """Projection onto the unit simplex."""
  if np.sum(s) <=1 and np.all(s >= 0):
      return s

  u = np.sort(s)[::-1]
  cssv = np.cumsum(u)
  rho = np.nonzero(u * np.arange(1, len(u)+1) > (cssv - 1))[0][-1]
  theta = (cssv[rho] - 1) / (rho + 1.0)

  return np.maximum(s-theta, 0)

def scaled_proj(c):
  sol = simplex_projection(np.abs(c)/delta)
  sol = delta*np.sign(c)*sol
  return sol

def proj(C1, C2):
  V1 = np.zeros_like(C1)
  for j in range(n_new):
      V1[:, j] = scaled_proj(C1[:, j])
  V2 = C2 / np.maximum(np.linalg.norm(C2, axis=0),1)
  return V1, V2

def clmo(C1, C2, A,b):
    V = cp.Variable((m, p))
    sub = cp.Problem(cp.Minimize(cp.trace(C2.T@V)), [cp.trace(A.T@V) <= b]+[cp.norm(V[:, j]) <= 1 for j in range(p)])
    sub.solve()
    V2 = V.value


    V1 = np.zeros_like(C1)
    for j in range(n_new):
        idx = np.argmax(np.abs(C1[:, j]))  # Find the index of the max gradient
        V1[idx, j] = -delta * np.sign(C1[idx, j])
    return V1, V2


In [ ]:
def irscg(init, budget, i):
  filename = 'irscg4_{}.csv'.format(i)
  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []

  oracle = 0
  count = 1

  t = 0
  X, D = init
  varsigma = 0.1

  ZX = 0
  ZD = 0
  s = 0
  elapsed_time = 0
  while elapsed_time <= budget:

    start = time.time()

    alpha = (t+1)**(-6/7)
    sigma = varsigma*(t+1)**(-2/7)

    xi    = np.random.randint(0, n_old-1, size=1)
    theta = np.random.randint(0, n_new-1, size=1)


    if t == 0:

      hat_gradXf= gradXf(X, D, theta)
      hat_gradDf= gradDf(X, D, theta)
      hat_gradg = gradg(D, xi)

    else:

      hat_gradXf = (1-alpha)*hat_gradXf + gradXf(X, D, theta) - (1-alpha)*gradXf(X_prev, D_prev, theta)
      hat_gradDf = (1-alpha)*hat_gradDf + gradDf(X, D, theta) - (1-alpha)*gradDf(X_prev, D_prev, theta)
      hat_gradg = (1-alpha)*hat_gradg +  gradg(D, xi)   - (1-alpha)* gradg(D_prev, xi)

    VX = lmoX(hat_gradXf)
    VD = lmoD(sigma*hat_gradDf +hat_gradg)
    X_prev = X
    D_prev = D
    X = X + alpha*(VX-X)
    D = D + alpha*(VD-D)

    s_prev = s
    s = s+(t+1)*sigma
    ZX = (s_prev*ZX-(t+1)*t*sigma*X_prev+(t+2)*(t+1)*sigma*X)/s
    ZD = (s_prev*ZD-(t+1)*t*sigma*D_prev+(t+2)*(t+1)*sigma*D)/s
    t += 1

    end = time.time()

    elapsed_time += end-start

    oracle += 2
    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(D,range( n_old))-g_opt)
      outer.append(f(X, D, range( n_old)))
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1
  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)


def irfscg(init, budget, i):
  filename = 'irfscg4_{}.csv'.format(i)
  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []
  oracle = 0
  count = 1
  n = n_old
  t = 0
  q = np.floor(n**0.5)
  S = q

  X,D = init
  ZX = 0
  ZD = 0
  s = 0
  w = 3/4
  varsigma = 0.1

  elapsed_time = 0
  while elapsed_time <= budget:

    start = time.time()
    if t < q:
      alpha = np.log(q+1)/(q+1)
    else:
      alpha = (t+1)**(-3/4)

    sigma = varsigma*(max(t,q+1)+1)**(-0.5)

    if t%q == 0:
      hat_gradXf = gradXf(X,D, range(n))
      hat_gradDf = gradDf(X,D, range(n))
      hat_gradg = gradg(D, range(n))

    else:
      ind1 = np.random.randint(0, n-1, size=int(S))
      ind2 = np.random.randint(0, n-1, size=int(S))

      hat_gradXf = hat_gradXf + gradXf(X,D,ind1) - gradXf(X_prev, D_prev, ind1)
      hat_gradDf = hat_gradDf + gradDf(X,D,ind1) - gradDf(X_prev, D_prev, ind1)
      hat_gradg = hat_gradg + gradg(D , ind2) - gradg(D_prev,ind2)

    VX = lmoX(hat_gradXf)
    VD = lmoD(sigma*hat_gradDf +hat_gradg)
    X_prev = X
    D_prev = D
    X = X + alpha*(VX-X)
    D = D + alpha*(VD-D)

    s_prev = s
    s = s+(t+1)*sigma
    ZX = (s_prev*ZX-(t+1)*t*sigma*X_prev+(t+2)*(t+1)*sigma*X)/s
    ZD = (s_prev*ZD-(t+1)*t*sigma*D_prev+(t+2)*(t+1)*sigma*D)/s
    t += 1

    end = time.time()

    elapsed_time += end-start
    if (t-1)%q == 0:
      oracle += 2*n
    else:
      oracle += 2*S

    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(D,range( n_old))-g_opt)
      outer.append(f(X, D, range( n_old)))
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1
  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

In [ ]:
def sbcgi(init,budget, i):
  filename = 'sbcgi4_{}.csv'.format(i)
  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []


  count = 1
  n  = n_old

  X, D,  oracle, elapsed_time = spiderfw(init, 100)
  g0 = g(D,range(n))


  t = 0
  while elapsed_time <= budget:

    start = time.time()


    alpha = 0.1/(t+1)**(2/3)

    xi    = np.random.randint(0, n-1, size=1)
    theta = np.random.randint(0, n-1, size=1)


    if t == 0:

      hat_gradXf = gradXf(X, D, theta)
      hat_gradDf = gradDf(X, D, theta)
      hat_gradg = gradg(D, xi)
      hat_g     = g(D,xi)

    else:

      hat_gradXf = (1-alpha)*hat_gradXf + gradXf(X, D, theta) - (1-alpha)*gradXf(X_prev, D_prev, theta)
      hat_gradDf = (1-alpha)*hat_gradDf + gradDf(X, D, theta) - (1-alpha)*gradDf(X_prev, D_prev, theta)
      hat_gradg = (1-alpha)*hat_gradg +  gradg(D, xi)   - (1-alpha)* gradg(D_prev, xi)
      hat_g     = (1-alpha)* hat_g    +    g(D, xi)     - (1-alpha)*   g(D_prev, xi)

    VX, VD = clmo(hat_gradXf, hat_gradDf, hat_gradg, np.trace(hat_gradg.T@D)+g0-hat_g+0.01/(t+1)**(1/3))
    X_prev = X
    D_prev = D
    X = X + alpha*(VX-X)
    D = D + alpha*(VD-D)
    t += 1

    end = time.time()

    elapsed_time += end-start
    oracle += 2
    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(D,range( n_old))-g_opt)
      outer.append(f(X, D, range( n_old)))
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1
  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

def sbcgf(init, budget, i):
  filename = 'sbcgf4_{}.csv'.format(i)
  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []

  oracle = 0
  count = 1

  n = n_old

  t = 0
  q = np.floor(n**0.5)
  S = q

  X, D, oracle, elapsed_time = spiderfw(init, 10**5)
  g0 = g(D, range(n))

  t = 0
  while elapsed_time <= budget:

    start = time.time()

    alpha = 10**(-3)


    if t%q == 0:
      hat_gradXf = gradXf(X,D, range(n))
      hat_gradDf = gradDf(X,D, range(n))
      hat_gradg = gradg(D, range(n))
      hat_g = g(D,range(n))

    else:
      ind1 = np.random.randint(0, n-1, size=int(S))
      ind2 = np.random.randint(0, n-1, size=int(S))
      #print(hat_gradXf.shape, gradXf(X,D,ind1).shape, gradXf(X_prev, D_prev, ind1).shape)
      hat_gradXf = hat_gradXf + gradXf(X,D,ind1) - gradXf(X_prev, D_prev, ind1)
      hat_gradDf = hat_gradDf + gradDf(X,D,ind1) - gradDf(X_prev, D_prev, ind1)
      hat_gradg = hat_gradg + gradg(D, ind2) - gradg(D_prev,ind2)
      hat_g     = hat_g + g(D, ind2) - g(D_prev, ind2)



    VX, VD = clmo(hat_gradXf, hat_gradDf, hat_gradg, np.trace(hat_gradg.T@D)+g0-hat_g+0.01/(t+1)**(1/3))
    X_prev = X
    D_prev = D
    X = X + alpha*(VX-X)
    D = D + alpha*(VD-D)
    t += 1

    end = time.time()

    elapsed_time += end-start
    #print('Iteration: {}, inner-level in x: {}, outer-level in x: {}'.format(t,g(D,range(n_old)),f(X,D,range(n_new))))
    if (t-1)%q == 0:
      oracle += 2*n
    else:
      oracle += 2*S

    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(D,range( n_old))-g_opt)
      outer.append(f(X, D, range( n_old)))
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1
  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

In [ ]:
def aripseg(init, budget, short, i):
  if short == True:
    gamma0 = 10**(-4)
    filename = 'aripseg_short4_{}.csv'.format(i)
  else:
    gamma0 = 10**(-2)
    filename = 'aripseg_long4_{}.csv'.format(i)

  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []

  oracle = 0
  count = 1
  t = 0
  X, D = init
  n = n_old
  rho0 = 1
  Gamma = 0
  r = 0.5
  elapsed_time = 0
  while elapsed_time <= budget:

    start = time.time()

    gamma = gamma0/(t+1)**0.75
    rho = rho0*(t+1)**0.25

    xi    = np.random.randint(0, n-1, size=1)
    theta = np.random.randint(0, n-1, size=1)


    hat_gradXf = gradXf(X, D,theta)
    hat_gradDf = gradDf(X, D,theta)
    hat_gradg = gradg(D, xi)

    #Need to work on
    YX, YD = proj(X-gamma*hat_gradXf, D-gamma*(hat_gradDf+rho*hat_gradg))

    xi    = np.random.randint(0, n-1, size=1)
    theta = np.random.randint(0, n-1, size=1)

    hat_gradXf = gradXf(YX, YD,theta)
    hat_gradDf = gradDf(YX, YD,theta)
    hat_gradg = gradg(YD,xi)

    X, D = proj(X-gamma*hat_gradXf, D-gamma*(hat_gradDf+rho*hat_gradg))

    Gamma_prev = Gamma
    Gamma = Gamma+(gamma*rho)**r

    if t == 0:
      bar_YX = X
      bar_YD = D
    else:
      bar_YX = (Gamma_prev*bar_YX+((gamma*rho)**r)*YX)/Gamma
      bar_YD = (Gamma_prev*bar_YD+((gamma*rho)**r)*YD)/Gamma
    t += 1
    end = time.time()

    elapsed_time += end-start
    #print('Iteration: {}, inner-level: {}, outer-level: {}'.format(t,g(bar_YD,range(n)),f(bar_YX, bar_YD,range(n))))
    oracle += 4
    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(bar_YD,range(n))-g_opt)
      outer.append(f(bar_YX, bar_YD,range(n)))
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1
  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

def dbgdsto(init, budget,short, i):

  if short == True:
    gamma = 5*10**(-3)
    filename = 'dbgdsto_short4_{}.csv'.format(i)
  else:
    gamma = 10**(-2)
    filename = 'dbgdsto_long4_{}.csv'.format(i)

  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []

  oracle = 0
  count = 1
  t = 0
  X,D = init
  alpha = 100
  beta = 100
  n = n_old

  elapsed_time = 0
  while elapsed_time <= budget:

    start = time.time()

    xi    = np.random.randint(0, n-1, size=1)
    theta = np.random.randint(0, n-1, size=1)


    hat_gradXf = gradXf(X, D,theta)
    hat_gradDf = gradDf(X, D,theta)
    hat_gradg = gradg(D,xi)
    hat_g = g(D,xi)

    norm_sq = np.linalg.norm(hat_gradg)**2
    hat_phi = min(alpha*(hat_g), beta*np.linalg.norm(norm_sq)**2)
    nu = max((hat_phi-np.trace(hat_gradDf.T@hat_gradg))/norm_sq, 0)

    #Need to work on
    X, D = proj(X -gamma*hat_gradXf,D -gamma*(hat_gradDf+nu*hat_gradg))

    t += 1
    end = time.time()

    elapsed_time += end-start
    #print('Iteration: {}, inner-level: {}, outer-level: {}'.format(t,g(D,range(n)),f(X, D, range(n))))
    oracle += 2
    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(D,range(n))-g_opt)
      outer.append(f(X, D, range(n)))
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1

  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

In [ ]:
def spiderfw(init, budget):
  X, D = init
  t = 0
  n = n_old
  q = np.ceil(np.sqrt(n))
  S = q
  oracle = 0
  time_elapsed = 0
  while t <= budget:
   start = time.time()
   eta = 0.1/(t+2)
   if t%q == 0:
      ind = range(n)
      hat_gradg = gradg(D, ind)
      oracle += n
   else:
      ind = np.random.randint(0, n-1, size=int(S))
      hat_gradg = hat_gradg + gradg(D,ind) - gradg(D_prev,ind)
      oracle += S

   VD = lmoD(hat_gradg)
   D_prev = D
   D = D+eta*(VD-D)

   t += 1
   end = time.time()
   time_elapsed += end-start
   #print('iteration: {}, inner-level: {}'.format(t, g(D,range(int(n)))))
   if (t-1)%q == 0:
      oracle += n
   else:
      oracle += S
  return X, D, oracle, time_elapsed




In [ ]:
# D_init = np.random.randn(m, p)
# D_init = D_init / np.linalg.norm(D_init, axis=0)  # Normalize atoms
# X_init = np.random.randn(p, n_old)
# X_init = X_init / np.sum(np.abs(X_init), axis=0, keepdims=True)
# init = (X_init, D_init)

In [ ]:
budget = 60
for i in range(0,10):
  np.random.seed(i)
  D_init = np.random.randn(m, p)
  D_init = D_init / np.linalg.norm(D_init, axis=0)  # Normalize atoms
  X_init = np.random.randn(p, n_old)
  X_init = X_init / np.sum(np.abs(X_init), axis=0, keepdims=True)
  init = (X_init, D_init)


  irscg(init, budget,i)
  irfscg(init, budget,i)
  sbcgi(init,budget,i)
  sbcgf(init,budget,i)
  aripseg(init, budget, True,i)
  aripseg(init, budget, False,i)
  dbgdsto(init, budget, True,i)
  dbgdsto(init, budget, False,i)